In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import time

os.environ["PREFECT__FLOWS__CHECKPOINTING"] = "true"

from prefect import task, Flow, Parameter
from prefect.executors import DaskExecutor
import xarray as xr

In [ ]:
import fsspec
import os

connection_string = os.environ.get("AZURE_STORAGE_CONNECTION_STRING")

fs = fsspec.filesystem("az", connection_string=connection_string)

# # # fs.ls('flow-outputs/')

# # fs.rm('flow-outputs/intermediate/bias_corrected_gcm',recursive=True)
# # fs.rm('flow-outputs/intermediate/funnel_metadata_store/bias_corrected_gcm',recursive=True)
# # fs.rm('flow-outputs/intermediate/bias_corrected_obs',recursive=True)
# # fs.rm('flow-outputs/intermediate/funnel_metadata_store/bias_corrected_obs',recursive=True)
# fs.rm('flow-outputs/intermediate/interpolated_obs',recursive=True)
# fs.rm('flow-outputs/intermediate/rechunked_obs/ERA5_1980_2020_pr_tasmax_tasmin_full_space.zarr', recursive=True)
# fs.rm('flow-outputs/intermediate/coarsened_obs/ERA5_1980_2020_pr_tasmax_tasmin_full_space_181x360_gridsize_10_10_llcorner_-90_0.zarr', recursive=True)
# fs.rm('flow-outputs/intermediate/coarsened_obs/ERA5_1980_2020_pr_tasmax_tasmin_full_space_362x720_gridsize_5_5_llcorner_-90_0.zarr', recursive=True)
# fs.rm('flow-outputs/intermediate/coarsened_obs/ERA5_1980_2020_pr_tasmax_tasmin_full_space_91x180_gridsize_20_20_llcorner_-90_0.zarr', recursive=True)

In [ ]:
# import dask
# from dask.distributed import Client
# from dask_gateway import Gateway

# client = Client(n_workers=4)
# client

In [ ]:
# executor = DaskExecutor("tcp://127.0.0.1:42329")
# from cmip6_downscaling.config.config import cache_store
# cache_store = CacheStore('/home/jovyan/shared-readwrite/gard-test-1/')

In [ ]:
run_hyperparameters = {
    "OBS": "ERA5",
    "GCM": "MIROC6",
    "SCENARIO": "ssp370",
    "TRAIN_PERIOD_START": "1991",
    "TRAIN_PERIOD_END": "1991",
    "PREDICT_PERIOD_START": "2079",
    "PREDICT_PERIOD_END": "2079",
    "VARIABLES": ["tasmax", "tasmin", "pr"],
    "BIAS_CORRECTION_METHOD": "quantile_transform",
    "BIAS_CORRECTION_KWARGS": None,
    "LABEL": "tasmax",
    "MODEL_TYPE": "AnalogRegression",
    "MODEL_PARAMS": {"n_analogs": 50},  # 'thresh': 0 for precip
}

In [ ]:
from cmip6_downscaling.workflows.gard_flow import gard_flow

In [ ]:
gard_flow.run(parameters=run_hyperparameters)

In [ ]:
ds = xr.open_zarr(
    "az://flow-outputs/intermediate/gard_pred_output/MIROC6_ssp370_1991_1991_2079_2079_tasmax_tasmin_pr_quantile_transform_AnalogRegression_tasmax.zarr"
)

In [ ]:
ds.prediction_error.isel(time=0).plot()